In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Net(nn.Module):

    def __init__(self, img_spat_dim, w1, pool1window, w2, pool2window, fc1dim, fc2dim, fc3dim):
        super(Net, self).__init__()
        self.pool1window_ = pool1window
        self.pool2window_ = pool2window
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(*w1)
        img_spat_dim = [(d - w1[-1] + 1) // pool1window for d in img_spat_dim] # 14*14
        
        self.conv2 = nn.Conv2d(*w2)
        img_spat_dim = [(d - w1[-1] + 1) // pool1window for d in img_spat_dim] # 5*5

        
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(img_spat_dim[0]*img_spat_dim[1]*w2[1], fc1dim)
        
        self.fc2 = nn.Linear(fc1dim, fc2dim)

        self.fc3 = nn.Linear(fc2dim, fc3dim)

        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), self.pool1window_)
        
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), self.pool2window_)
        
        # flatten x so its shape is BS*Feats
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net([32,32], [1, 6, 5], 2, [6, 16, 5], 2, 120, 84, 10)
print(net)


In [ ]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

In [ ]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)


In [ ]:
# TF will take in a 4D Tensor of [nSamples x Height x Width x nChannels]


In [1]:
import tensorflow as tf
import numpy as np

In [ ]:
a = tf.constant(2.0)
b = tf.constant(3.0)
c = a * b

In [ ]:
sess = tf.Session()
sess.run(c)

In [ ]:
sess.close()

In [3]:
def network(x, weights, biases, name="network"):
    # 1. Hidden layer, ReLU
    layer_1 = tf.add(tf.matmul(x, weights["h1"]), biases["b1"])
    layer_1 = tf.nn.relu(layer_1)

    # 2. Hidden layer, ReLU
    layer_2 = tf.add(tf.matmul(layer_1, weights["h2"]), biases["b2"])
    layer_2 = tf.nn.relu(layer_2)

    # Output layer with linear activation
    out_layer = tf.add(tf.matmul(layer_2, weights["out"]), biases["bout"])
    return out_layer

#pred = network(x, weights, biases, name="network")

In [4]:
test_image = np.random.randn(900,1)

with tf.Session() as sess:
    saver = tf.train.import_meta_graph("my_model.meta")
    saver.restore(sess, tf.train.latest_checkpoint("/dir/to/model"))
    op_to_restore = graph.get_tensor_by_name("network:0")

OSError: File my_model.meta does not exist.